In [31]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [32]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [33]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [34]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [35]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [36]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [37]:
df['loc']

1     (7153 Fisher Island Dr #7153, Miami Beach, FL ...
2     (1425 Brickell Ave, Miami, FL 33131, USA, (25....
3     (50 South Pointe Dr APT 3402, Miami Beach, FL ...
4     (1800 Sunset Harbour Dr, Miami Beach, FL 33139...
5     (10201 Collins Ave #1603s, Bal Harbour, FL 331...
6     (100 South Pointe Dr APT 3605, Miami Beach, FL...
7     (360 Ocean Dr #1106s, Key Biscayne, FL 33149, ...
8     (2821 S Bayshore Dr b, Miami, FL 33133, USA, (...
9     (13621 Deering Bay Dr, Coral Gables, FL 33158,...
10    (3535 Hiawatha Ave APT 504, Miami, FL 33133, U...
Name: loc, dtype: object

In [38]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [39]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Palazo\r7153 Fisher Island Dr 7153\rMiami Beach,5/22/2021,2/2/2022,183,"$21,000,000.00",7308,"$2,873.56",Erena Stus,Elena Bluntzer,Palazo 7153 Fisher Island Dr 7153 Miami Beach,Palazo,7153 Fisher Island Dr 7153 Miami Beach,"(7153 Fisher Island Dr #7153, Miami Beach, FL ...","(25.762737, -80.13597, 0.0)",25.762737,-80.135970,0.0
2,Four Seasons Residences\r1425 Brickell Ave PH1...,11/4/2021,1/31/2022,41,"$15,900,000.00",8805,"$1,805.79",Elena Bluntzer,Jonathan Mann,Four Seasons Residences 1425 Brickell Ave PH1 ...,Four Seasons Residences,1425 Brickell Ave PH1 Miami,"(1425 Brickell Ave, Miami, FL 33131, USA, (25....","(25.7591323, -80.1918295, 0.0)",25.759132,-80.191829,0.0
3,Continuum South Beach\r50 S Point Dr 3402\rMia...,10/3/2021,2/1/2022,82,"$11,700,000.00",4246,"$3,861.39",Eloy Carmenate,Kevin Crego,Continuum South Beach 50 S Point Dr 3402 Miami...,Continuum South Beach,50 S Point Dr 3402 Miami Beach,"(50 South Pointe Dr APT 3402, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0
4,Sunset Harbour South\r1800 Sunseet Harbour Dr ...,5/4/2020,1/31/2022,578,"$7,850,000.00",10732,"$1,109.54",Dina Goldentayer,Bryan Sereny PA,Sunset Harbour South 1800 Sunseet Harbour Dr T...,Sunset Harbour South,1800 Sunseet Harbour Dr TS-2/3 Miami Beach,"(1800 Sunset Harbour Dr, Miami Beach, FL 33139...","(25.7944878, -80.14480089999999, 0.0)",25.794488,-80.144801,0.0
5,Oceana Bal Harbour Condo\r10201 Collins Ave 16...,10/28/2021,2/4/2022,79,"$7,050,000.00",3264,"$2,159.93",Nathalie Azoubel,Tania De Jesus PA,Oceana Bal Harbour Condo 10201 Collins Ave 160...,Oceana Bal Harbour Condo,10201 Collins Ave 1603S Bal Harbour,"(10201 Collins Ave #1603s, Bal Harbour, FL 331...","(25.8951677, -80.12315, 0.0)",25.895168,-80.123150,0.0
6,Continuum South Beach\r100 S Point Dr 3605\rMi...,9/2/2021,1/31/2022,131,"$4,975,000.00",1793,"$2,774.68",Jorge Martinez,Selen Arditi,Continuum South Beach 100 S Point Dr 3605 Miam...,Continuum South Beach,100 S Point Dr 3605 Miami Beach,"(100 South Pointe Dr APT 3605, Miami Beach, FL...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
7,Oceana Key Biscayne\r360 Ocean Dr 1106S\rKey B...,10/29/2021,1/31/2022,31,"$4,775,000.00",2257,"$2,115.64",Claudine Coto,Elaine Kauffmann,Oceana Key Biscayne 360 Ocean Dr 1106S Key Bis...,Oceana Key Biscayne,360 Ocean Dr 1106S Key Biscayne,"(360 Ocean Dr #1106s, Key Biscayne, FL 33149, ...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0
8,Two Park Grove Condo\r2821 S Bayshore Dr PH-B\...,1/19/2022,2/1/2022,1,"$4,450,000.00",2556,"$1,741.00",Carole Smith,Carole Smith,Two Park Grove Condo 2821 S Bayshore Dr PH-B C...,Two Park Grove Condo,2821 S Bayshore Dr PH-B Coconut Grove,"(2821 S Bayshore Dr b, Miami, FL 33133, USA, (...","(25.7283709, -80.238272, 0.0)",25.728371,-80.238272,0.0
9,Deering Bay\r13621 Deering Bay Dr 104/204\rCor...,10/25/2021,1/31/2022,51,"$4,450,000.00",6576,$676.70,Dennis Carvajal,MariaClaudia Fernandez,Deering Bay 13621 Deering Bay Dr 104/204 Coral...,Deering Bay,13621 Deering Bay Dr 104/204 Coral Gables,"(13621 Deering Bay Dr, Coral Gables, FL 33158,...","(25.637681, -80.290989, 0.0)",25.637681,-80.290989,0.0
10,Residences at Vizcaya\r3535 Hiawatha Ave 504\r...,5/20/2021,2/1/2022,203,"$4,425,000.00",4000,"$1,106.25",Juliana Savoia PA,Jorge Uribe,Residences at Vizcaya 3535 Hiawatha Ave 504 Miami,Residences at Vizcaya,3535 Hiawatha Ave 504 Miami,"(3535 Hiawatha Ave APT 504, Miami, FL 33133, U...","(25.7405538, -80.2189503, 0.0)",25.740554,-80.218950,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [46]:
df.at[1,'building_name']=('Palazzo Del Mare')
df.at[1,'Agent']=('Alex Filkshteyn')
df.at[1,'Buyer Agent']=('Elena Bluntzer')

df.at[2,'Buyer Agent']=('Jonathan Mann')
df.at[2,'Agent']=('Elena Bluntzer & Jonathan Garcia')

In [47]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [48]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 30th - February 5th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [49]:
m.save('index.html')